In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "EXAMPLE"

# LCEL 다루는 방법

# RunnableParallel : 입력 및 출력 조작
##### RunnableParallel은 시퀀스에서 다음 Runnable 입력 형식과 일치하도록 하나의 Runnable의 출력을 조작한다.

In [ ]:
!pip install --upgrade --quiet  langchain langchain-openai

In [ ]:
!pip install faiss-gpu

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ['천준석은 인천에서 일을 한다.'], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt | model | StrOutputParser()
)

retrieval_chain.invoke("천준석은 어디서 일하니?")

'인천'

## 약어로 itemgetter 사용
##### RunnableParallel와 결합하여 데이터를 추출하기 위해 python 약어를 사용할 수 있다.
##### itemgetter를 사용하여 맵에서 특정 키를 추출한다.

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["천준석은 인천에서 일을 한다."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt | model | StrOutputParser()
)

chain.invoke({"question": "천준석은 어디서 일하니?", "language": "english"})

'Cheon Jun-seok works in Incheon.'

## 병렬화
##### RunnableParallel 을 사용하면 여러 Runnable을 병렬로 쉽게 실행하고 이러한 Runnable의 출력을 반환할 수 있다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("{주제}에 대해서 농담을 말해줘") | model
poem_chain = (
    ChatPromptTemplate.from_template("{주제}에 대해서 2줄 정도의 시를 써줘") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"주제":"곰"})

{'joke': AIMessage(content='곰이 왜 항상 참견을 하는가?\n\n곰곰히 생각하면 당연하다!', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 23, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None}),
 'poem': AIMessage(content='숲속 깊은 곳에서 뚜벅뚜벅 걸어오는\n귀여운 곰이 나를 보며 웃는다.', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 26, 'total_tokens': 71}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})}

In [ ]:
%%timeit

joke_chain.invoke({"주제": "곰"})

1.37 s ± 358 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

poem_chain.invoke({"주제": "곰"})

1.24 s ± 335 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

map_chain.invoke({"주제": "곰"})

1.49 s ± 406 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### 두 개를 동시 실행 한다해도 동일하거나 더 빠른 런타임을 갖는 것을 볼 수 있다.

# RunnablePassthrough : 데이터 전달
##### RunnablePassthrough를 사용하면 변경 없이 또는 추가 키를 추가하여 입력을 전달할 수 있다. 일반적으로 RunnableParallel과 함께 사용되어 맵의 새 키에 데이터를 할당한다.
##### 자체적으로 호출되는 RunnablePassthrough()는 입력을 받아 통과시킨다.
##### RunnablePassthrough는 RunnablePassthrough.assign 으로 호출된 입력을 취하고 할당 함수에 전달된 추가 인수를 추가한다.

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

## 검색
##### RunnableMap과 RunnablePassthrough를 함께 사용하는 예제는 보자

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
vectorstore = FAISS.from_texts(
    ["천준석은 김치를 먹는다."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("천준석은 뭐를 먹니?")

'김치'

# RunnableLambda : 사용자 정의 함수 실행
##### 이러한 함수는 모든 입력이 SINGLE 인수여여 하고, 여러 인수를 허용하는 함수가 있는 경우 단일 입력을 허용하고 압축 해제하는 래퍼를 작성해야 한다.

In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [ ]:
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 = 12', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

## 실행 가능한 내용
##### 실행 가능한 람다는 콜백, 태그 및 기타 구성 정보를 중천된 실행에 전달하는데 사용할 수 있는 RunnableConfig를 선택적으로 허용한다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig

In [ ]:
import json


def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 62
	Prompt Tokens: 56
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $9.6e-05


# RunnableBranch : 입력을 기반으로 로직을 동적으로 라우팅
##### 라우팅을 사용하면 LLM과의 상호 작용에 대한 구조와 일관성을 제공한다.
##### 수행 방법으로는 2가지가 있다.


1.   RunnableLambda : 조건부로 실행 파일 반환 (선호)
2.  RunnableBranch



##### 먼저 들어오는 질문을 langchain, openai, other 로 구분하여 식별하는 체인을 만들어보자.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `openai`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain.invoke({"question": "how do I call openai?"})

'openai'

In [ ]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)
openai_chain = (
    PromptTemplate.from_template(
        """You are an expert in openai. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)

##### 사용자 정의 기능을 사용하여 서로 다른 출력 간의 라우팅을 해보자

In [ ]:
def route(info):
    if "openai" in info["topic"].lower():
        return openai_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [ ]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [ ]:
full_chain.invoke({"question": "how do I use openai?"})

AIMessage(content='As Dario Amodei told me, you can use OpenAI by signing up for an API key on their website and following the documentation to integrate their models into your applications.', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 47, 'total_tokens': 83}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, to use LangChain, you need to first create an account on the platform and then start exploring the various language learning features it offers. You can practice speaking, listening, reading, and writing in your target language, and track your progress along the way. Additionally, you can connect with language tutors and other learners to further enhance your language skills.', response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 44, 'total_tokens': 119}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='2 + 2 equals 4.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 24, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

## RunnableBranch 사용
##### 입력을 기반으로 실행할 조건 및 실행 가능 항목 세트를 정의할 수 있는 특수한 유형의 방법입니다. 그런데, 사용자 정의 함수에서 모두 제공하기에 사용자 정의 함수를 쓰는 것을 선호됩니다.

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "openai" in x["topic"].lower(), openai_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch
full_chain.invoke({"question": "how do I use openai?"})

AIMessage(content="As Dario Amodei told me, to use OpenAI, you can start by signing up for an API key on their website. This key will allow you to access their various language models and tools for natural language processing and generation. You can then use this API key to integrate OpenAI's technology into your own applications or projects.", response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 47, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, to use LangChain, you must first create an account on the platform and then begin by selecting the language you wish to learn. From there, you can access a variety of language learning tools, including lessons, exercises, and conversation practice. It is important to regularly practice and engage with the material to effectively improve your language skills.', response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 44, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='4', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 24, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

# 런타임 인수 바인딩
##### Runnable.bind()를 사용해서 상수 인수를 사용하여 Runnable 시퀀스 내에서 Runnable을 호출할 수 있다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)

model = ChatOpenAI(temperature=0)
runnable = (
    {
        "equation_statement": RunnablePassthrough()
    }
    | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION: 
Subtract 7 from both sides:
x^3 = 5

Take the cube root of both sides:
x = ∛5


##### stop 단어로 모델을 호출할 수 있다.

In [ ]:
runnable = (
    {"equation_statement" : RunnablePassthrough()}
    | prompt | model.bind(stop="SOLUTION") | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12




## OpenAI
##### 바인딩의 특히 응용 프로그램 중 하나인 OpenAI 기능을 호환되는 OpenAI 모델에 연결하는 것이다.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo").bind(tools=tools)
model.invoke("Waht's the weather in South Korea, Incheon")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CVCxg3bhs1DU9IpF96sMh377', 'function': {'arguments': '{"location":"Incheon, South Korea"}', 'name': 'get_current_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 85, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'tool_calls', 'logprobs': None})

# 런타임에 체인 내부 구성
##### 작업을 수행하는 여러 가지 다른 방법을 실험하거나 사용자에게 노출시키는 방법이 있다.


1.   configurable_fields : 실행 가능 항목의 특정 필드를 구성한다.
2.   configurable_alternatives : 런타임 중에 설정할 수 있는 특정 실행 가능 항목 대안을 나열한다.



## configurable_fields
### LLM
##### LLM을 사용하면 온도와 같은 항목을 구성할 수 있다.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM"
    )
)

In [ ]:
model.invoke("숫자 하나만 무작위로 말해바~")

AIMessage(content='7', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 24, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("숫자 하나만 무작위로 말해바~")

AIMessage(content='5', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 24, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

##### 체인을 일부로 해당 작업을 수행할 수 있다.

In [ ]:
prompt = PromptTemplate.from_template("{x}보다 큰 숫자 하나만 골라바")
chain = prompt | model

In [ ]:
chain.invoke({"x": 15})

AIMessage(content='7', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 18, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
chain.with_config(configurable={"llm_temperature": 0.9}).invoke({"x": 7})

AIMessage(content=' 3', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 18, 'total_tokens': 20}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

## HubRunnables 사용
##### 프롬프트 전환에 유용하다.

In [ ]:
!pip install langchainhub

In [ ]:
from langchain.runnables.hub import HubRunnable

In [ ]:
prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

In [ ]:
prompt.invoke({"question": "foo", "context": "bar"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: foo \nContext: bar \nAnswer:")])

In [ ]:
prompt.with_config(configurable={"hub_commit": "rlm/rag-prompt-llama"}).invoke(
    {"question": "foo", "context": "bar"}
)

ChatPromptValue(messages=[HumanMessage(content="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: foo \nContext: bar \nAnswer: [/INST]")])

## configurable_alternatives
### LLM

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0).configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key="anthropic",
    openai=ChatOpenAI(),
)
prompt = PromptTemplate.from_template("{주제}에 대해서 농담 하나 말해바")
chain = prompt | llm

In [ ]:
chain.invoke({"주제":"인천"})

AIMessage(content='인천 사람들은 바다가 가까워서 물 한 모금 마시면 다 바다 물이 된다던데요!', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 25, 'total_tokens': 68}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
chain.with_config(configurable={"llm": "openai"}).invoke({"주제": "곰"})

AIMessage(content='곰이 어떤 차에서 가장 잘 탈까요? \n\n포르쉐! 왜냐하면 포르쉐 ㅋㅋㅋ', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

## 프롬프트
##### 프롬프트를 번갈아 가며 수행할 수 있다.

In [ ]:
llm = ChatOpenAI(temperature=0)
prompt = PromptTemplate.from_template(
    "{주제}에 대해서 농담 해바"
).configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="joke",
    poem=PromptTemplate.from_template("{주제}에 대해서 짧은 시를 써줘"),
)
chain = prompt | llm

In [ ]:
chain.invoke({"주제": "곰"})

AIMessage(content='왜 곰이 항상 피곤한가요? \n\n- 당연히 자다가 깨어나면서 "곰이다" 하면서 피곤해요!', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 20, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
chain.with_config(configurable={"prompt": "poem"}).invoke({"주제": "곰"})

AIMessage(content='숲 속 깊은 곳에\n턱시도 높은 곰이 산다\n턱시도 높은 곰이\n숲 속 깊은 곳에 산다', response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 22, 'total_tokens': 79}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

## 프롬프트와 LLM 구성

In [ ]:
llm = ChatOpenAI(temperature=0).configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key="anthropic",
    openai=ChatOpenAI(),
)
prompt = PromptTemplate.from_template(
    "{주제}에 대해서 농담 해바"
).configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="joke",
    poem=PromptTemplate.from_template("{주제}에 대해서 짧은 시를 써줘"),
)
chain = prompt | llm

In [ ]:
chain.with_config(configurable={"prompt": "poem", "llm": "openai"}).invoke(
    {"주제": "곰"}
)

AIMessage(content='숲 속 깊은 곳에서\n큼지막한 곰이 살아요\n털부터 발끝까지 검은색\n둥실둥실한 몸집에\n우쭈쭈한 뒷발로\n뛰어놀며 놀아요\n\n숲속을 터벅터벅\n달려다니는 모습은\n아름다운 자연의 힘\n곰이라는 친구가\n우리 옆에서 함께\n살아가는 모습을 보면\n우린 행복을 느껴요.', response_metadata={'token_usage': {'completion_tokens': 174, 'prompt_tokens': 22, 'total_tokens': 196}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
chain.with_config(configurable={"llm": "openai"}).invoke({"주제": "곰"})

AIMessage(content='왜 곰은 항상 화내나요? \n\n- 왜냐하면 항상 곰곰이 생각하니까요!', response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 20, 'total_tokens': 60}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

# @chain 데코레이터를 사용하여 실해 가능 파일 만들기
##### @chain 데코레이터를 추가하여 임의의 함수를 체인으로 바꿀수있다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

In [ ]:
prompt1 = ChatPromptTemplate.from_template("{주제}에 대해서 농담 해줘")
prompt2 = ChatPromptTemplate.from_template("{농담} 해당 농담에 대한 주제가 뭐야")

In [ ]:
@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"주제": text})
    output1 = ChatOpenAI().invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | ChatOpenAI() | StrOutputParser()
    return chain2.invoke({"농담": parsed_output1})

In [ ]:
custom_chain.invoke("bears")

'곰 같이 친절한 사람들에 대한 농담입니다.'

# 폴백 추가
##### LLM API 문제, 잘못된 모델 출력, 기타 통합 문제 등 LLM 애플리케이션에는 실패할 수 있는 지점이 많기 때문에 폴백을 적절하게 사용하면 도움이 많이 된다.

## LLM API
##### LLM API 요청으로 인해 실패를 폴백을 사용하면 보호하는데 도움이 된다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatAnthropic

##### OpenAI에서 폴백을 포함한 LLM이 아래의 예이다

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"animal": "kangaroo"}))
    except RateLimitError:
        print("Hit error")

Hit error


# 사용자 정의 생성기 함수 스트림
##### LCEL 파이프라인에서 yield 생성기 함수를 사용할 수 있다.
##### 이러한 생성기의 서명은 Iterator[Input] -> Iterator[Output], 비동기 생성기의 경우 AsyncIterator[Input] -> AsyncIterator[Output]
##### 사용자 정의 출력 파서를 구현하거나 스트리밍 기능을 유지하면서 이전 단계의 출력을 수정할 때 용이하다.

## 동기

In [ ]:
from typing import Iterator, List

from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    "Write a comma-separated list of 5 animals similar to: {animal}"
)
model = ChatOpenAI(temperature=0.0)

str_chain = prompt | model | StrOutputParser()

In [ ]:
for chunk in str_chain.stream({"animal": "bear"}):
  print(chunk, end="", flush=True)

1. Wolf
2. Lion
3. Tiger
4. Gorilla
5. Panda

In [ ]:
str_chain.invoke({"animal": "bear"})

'1. Wolf\n2. Tiger\n3. Lion\n4. Gorilla\n5. Panda'

In [ ]:
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    buffer = ""
    for chunk in input:
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]

In [ ]:
list_chain = str_chain | split_into_list

In [ ]:
for chunk in list_chain.stream({"animal": "bear"}):
  print(chunk, flush=True)

['1. Wolf']
['2. Lion']
['3. Tiger']
['4. Gorilla']
['5. Panda']


## 비동기

In [ ]:
from typing import AsyncIterator


async def asplit_into_list(
    input: AsyncIterator[str],
) -> AsyncIterator[List[str]]:  # async def
    buffer = ""
    async for (
        chunk
    ) in input:  # `input` is a `async_generator` object, so use `async for`
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]


list_chain = str_chain | asplit_into_list

In [ ]:
async for chunk in list_chain.astream({"animal": "bear"}):
    print(chunk, flush=True)

['1. Wolf\n2. Tiger\n3. Lion\n4. Gorilla\n5. Panda']


In [ ]:
await list_chain.ainvoke({"animal": "bear"})

['1. Wolf\n2. Lion\n3. Tiger\n4. Gorilla\n5. Panda']

# 실행 파일 검사
##### LCEL을 사용하여 실행 가능 파일을 생성한 후에는 무슨 일이 일어나고 있는지 검사할 수 있다.

In [ ]:
!pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

In [ ]:
!pip install grandalf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 760.9 kB/s eta 0:00:00


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## 그래프 얻기

In [ ]:
# 실행 가능한 그래프를 얻는다.
chain.get_graph()

## 그래프 인쇄

In [ ]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

## 프롬프트 받기
##### 체인에 프롬프트가 표시되도록 한다.

In [ ]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])]

# 메시지 기록(메모리) 추가
##### RunnableWithMessageHistory를 통해 특정 유형의 체인에 메시지 기록을 추가할 수 있다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        # 기록 저장 홀더
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model

## 인 메모리
##### 채팅 기록이 메모리에 저장되는 예제를 보자

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='Cosine is a trigonometric function that gives the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 33, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
# 대화 내용을 기억한다
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='Cosine is a mathematical function that relates the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 101, 'total_tokens': 125}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
# 하지만 세션이 달라질 경우 기억하지 못한다
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "def234"}},
)

AIMessage(content='I can help with math problems! Just let me know what you need assistance with.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 30, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

##### 메시지 기록을 추적하는 구성 매개변수를 사용자 정의로 구성할 수 있다.

In [ ]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [ ]:
with_message_history.invoke(
    {"ability": "math", "input": "Hello"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

AIMessage(content='Hi! How can I assist you today with math or any other questions you may have?', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 29, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})

## 다양한 서명(입력)
##### dict

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": ChatOpenAI()})


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

with_message_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content='Simone de Beauvoir believed that human beings have free will and the ability to make choices and shape their own lives. She argued that individuals are not bound by pre-determined roles or social norms, and that they have the power to define themselves through their actions and choices. Beauvoir believed in the importance of personal responsibility and agency, and rejected the idea of determinism or predestination.', response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 17, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_fa89f7a861', 'finish_reason': 'stop', 'logprobs': None})}

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="How did this compare to Sartre")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content='Simone de Beauvoir\'s beliefs about free will were heavily influenced by existentialist philosophy, particularly the work of Jean-Paul Sartre. Both Beauvoir and Sartre believed in the concept of radical freedom, which means that individuals are ultimately responsible for their own actions and choices. They rejected the idea of determinism and believed that human beings have the capacity to create their own values and meaning in a world that lacks inherent purpose or meaning.\n\nHowever, there were some differences in their views on free will. Sartre emphasized the idea of "bad faith," which refers to individuals denying their own freedom and responsibility by conforming to societal expectations or norms. Beauvoir, on the other hand, focused more on the ways in which social structures and gender roles can limit individuals\' freedom and autonomy. She argued that women, in particular, face societal constraints that can restrict their ability to exerc

##### 메시지 입력

In [ ]:
RunnableWithMessageHistory(
    ChatOpenAI(),
    get_session_history,
)

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7ce21d381360>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7ce21d433670>, openai_api_key=SecretStr('**********'), openai_proxy=''), config_factories=[<function Runnable.with_listeners.<locals>.<lambda> at 0x7ce21d408ca0>]), config={'run_name': 'RunnableWithMessageHistory'}), get_session_history=<function get_session_history at 0x7ce21d386320>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [ ]:
from operator import itemgetter

RunnableWithMessageHistory(
    itemgetter("input_messages") | ChatOpenAI(),
    get_session_history,
    input_messages_key="input_messages",
)

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  input_messages: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=RunnableLambda(itemgetter('input_messages'))
  | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7ce21d431360>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7ce21d430e20>, openai_api_key=SecretStr('**********'), openai_proxy=''), config_factories=[<function Runnable.with_listeners.<locals>.<lambda> at 0x7ce21d40aef0>]), config={'run_name': 'RunnableWithMessageHistory'}), get_session_history=<function get_session_history at 0x7ce21d386320>, input_messages_key='input_messages', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=Tru

## 영구적인 저장
##### 대화를 영구적으로 저장할려면 공간이 필요하다. redis를 사용해보자

In [ ]:
!pip install --upgrade --quiet redis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 2.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

In [ ]:
with_message_history.invoke(
    {"ability": "math", "input": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)